# LAB12: Code Llama

Utilizaremos la API de `together.ai` para hacer inferencia a otros modelos.

La API **no es gratis** pero con cuentas nuevas nos dan 25$ de crédito y nos dejan utilizar modelos como Llama 2 y 3, Mistral,

Code Llama es un modelo de inteligencia artificial desarrollado por Meta, construido sobre la base de Llama 2 y afinado específicamente para generar y discutir código.



In [ ]:
import getpass
api_key = getpass.getpass("Enter your Together.AI API Key:")

Enter your Together.AI API Key:··········


## Creamos una función de orquestración

In [ ]:
import os
import warnings
import requests
import json
import time

url = "https://api.together.xyz/inference"
headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

def orchestration(prompt,
                  add_inst=True,
                  model="togethercomputer/CodeLlama-7b-Instruct",
                  temperature=0.0,
                  max_tokens=1024,
                  verbose=False,
                  base=2,
                  attempt=0,
                  max_tries=3):


    prompt = f"[INST]{prompt}[/INST]"

    if verbose:
        print(f"Prompt:\n{prompt}\n")
        print(f"Model: {model}")

    data = {
            "model": model,
            "prompt": prompt,
            "temperature": temperature,
            "max_tokens": max_tokens
        }

    # Calculate waiting time based on the current attempt number
    wait_seconds = base ** attempt

    try:
        response = requests.post(url, headers=headers, json=data)
        if response.status_code == 200:
            return response.json()['output']['choices'][0]['text']
        else:
            raise Exception(f"API response not successful: {response.status_code}")
    except Exception as e:
        if attempt < max_tries - 1:
            print(f"Error message: {e}")
            print(f"Response object: {response}")
            print(f"Attempt {attempt + 1}")
            print(f"Waiting {wait_seconds} seconds before retrying.")
            time.sleep(wait_seconds)
            return orchestration(prompt, add_inst, model, temperature, max_tokens, verbose, base, attempt + 1, max_tries)
        else:
            print(f"Tried {max_tries} times to make API call to get a valid response object.")
            print("Returning the last received response")
            return response

    return response


## Escribiendo código

In [ ]:
prompt = f"""
Create a Python function to find the smallest value in the list the largest value in the list
"""
response = orchestration(prompt, model="Qwen/Qwen2.5-Coder-32B-Instruct")
print(response)

Certainly! You can create a Python function that takes a list as input and returns both the smallest and the largest values in that list. Here's a simple implementation:

```python
def find_min_max(values):
    if not values:
        return None, None  # Return None for both if the list is empty

    min_value = min(values)
    max_value = max(values)
    
    return min_value, max_value

# Example usage:
numbers = [3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5]
min_val, max_val = find_min_max(numbers)
print(f"The smallest value is: {min_val}")
print(f"The largest value is: {max_val}")
```

### Explanation:
- The function `find_min_max` takes a list `values` as its parameter.
- It first checks if the list is empty. If it is, the function returns `None` for both the minimum and maximum values.
- If the list is not empty, it uses the built-in `min()` and `max()` functions to find the smallest and largest values in the list, respectively.
- Finally, it returns a tuple containing the smallest and larges

In [ ]:
def find_min_max(values):
    if not values:
        return None, None  # Return None for both if the list is empty

    min_value = min(values)
    max_value = max(values)

    return min_value, max_value

# Example usage:
numbers = [3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5]
min_val, max_val = find_min_max(numbers)
print(f"The smallest value is: {min_val}")
print(f"The largest value is: {max_val}")

The smallest value is: 1
The largest value is: 9


In [ ]:
numbers = [42, 52, 47, 47, 53, 48, 47, 53, 55, 56, 57, 50, 48, 45]

results = find_min_max(numbers)
print(results)

(42, 57)


## Resolviendo tareas

In [ ]:
prompt = """
Create a function that calculates the X fibonacci number.
"""

response = orchestration(prompt, verbose=True, model="Qwen/Qwen2.5-Coder-32B-Instruct")
print(response)

Prompt:
[INST]
Create a function that calculates the X fibonacci number.
[/INST]

Model: Qwen/Qwen2.5-Coder-32B-Instruct

Certainly! To calculate the \( n \)-th Fibonacci number, you can use several approaches, such as recursion, iteration, or dynamic programming. Here, I'll provide a simple and efficient iterative approach:

```python
def fibonacci(n):
    if n <= 0:
        raise ValueError("The input should be a positive integer.")
    elif n == 1:
        return 0
    elif n == 2:
        return 1

    a, b = 0, 1
    for _ in range(2, n):
        a, b = b, a + b
    return b

# Example usage:
n = 10
print(f"The {n}-th Fibonacci number is: {fibonacci(n)}")
```

### Explanation:
- **Base Cases**: 
  - If \( n = 1 \), the Fibonacci number is 0.
  - If \( n = 2 \), the Fibonacci number is 1.
- **Iterative Calculation**:
  - Start with the first two Fibonacci numbers, \( a = 0 \) and \( b = 1 \).
  - Iterate from 2 to \( n-1 \), updating \( a \) and \( b \) such that \( b \) becomes th

In [ ]:
def fibonacci(x):
    if x <= 0:
        raise ValueError("The input must be a positive integer.")
    elif x == 1:
        return 0
    elif x == 2:
        return 1

    a, b = 0, 1
    for _ in range(2, x):
        a, b = b, a + b
    return b

# Example usage:
x = 10
print(f"The {x}-th Fibonacci number is: {fibonacci(x)}")

The 10-th Fibonacci number is: 34


In [ ]:
fibonacci(35)

5702887

Esta implementación es algo pocha... Es mejor:

In [ ]:
def fibonacci_fast(n):
    a, b = 0, 1
    for i in range(n):
        a, b = b, a + b
    return a

In [ ]:
fibonacci_fast(35)

9227465

Vamos a comparar

In [ ]:
import time

n=40

start_time = time.time()
calculate_fibonacci(n)
end_time = time.time()
print(f"calculate_fibonacci({n}) ")
print(f"Time: {end_time-start_time}")

start_time = time.time()
fibonacci_fast(n)
end_time = time.time()
print(f"fibonacci_fast({n}) ")
print(f"Time: {end_time-start_time}")

calculate_fibonacci(40) 
Time: 47.27380609512329
fibonacci_fast(40) 
Time: 6.747245788574219e-05
